In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import os
from pathlib import Path
import numpy as np
import re

In [9]:
path = Path(input("请输入路径："))

In [35]:
os.chdir(path)
print(f"当前路径为:{os.getcwd()}")

当前路径为:/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230424鞋垫空放自重漂移测试


In [36]:
# 新建动态校准后文件夹
DynamicCalibreDir = "DynamicCalibre"
if not os.path.exists(DynamicCalibreDir):
    os.mkdir(DynamicCalibreDir)

In [37]:
# 动态零压校准代码
def SensorsCalibre(part_data,part):
    sensors_calibre = pd.DataFrame({})

    # 传感器列表
    sensors = ['p1-c', 'p2-c', 'p3-c', 'p4-c', 'p5-c', 'p6-c', 'p7-c',
       'p8-c', 'p9-c', 'p10-c', 'p11-c', 'p12-c']

    for sensor_name in sensors:
        sensor = part_data[sensor_name]

        # 查找峰值
        # 以快跑 16km/h ，对应至少间隔 22 个数
        peaks,_ = find_peaks(-sensor,-sensor.max(),width=20)
        L_shape = sensor.shape
        peakls = peaks.tolist()

        # 查找峰值并绘制图片
        #sensor.plot()
        #plt.plot(peaks*2,sensor.iloc[peakls])

        # 使用峰值校准数据
        # 所有数据减去前一个峰值
        # 第一个峰值之前的数据减去第一个峰值
        range_start = peakls[0]
        df_calibre = pd.DataFrame({})
        # 选取区间内数值
        snip_data= sensor.iloc[0:range_start]
        # 进行校准
        calibre_sensor_snip = snip_data.apply(lambda x: x-sensor.iloc[range_start])
        df_calibre = pd.concat([df_calibre.reset_index(drop=True), 
                                        calibre_sensor_snip.reset_index(drop=True)],axis=0)   
        for i in range(len(peakls)):

            # 选取两个波谷之间的区间
            range_start = peakls[i]
            if i < len(peakls)-1:
                range_end = peakls[i+1]
            else:
                range_end = L_shape[0]
            

            # 选取区间内数值
            snip_data= sensor.iloc[range_start:range_end]
            # 进行校准
            calibre_sensor_snip = snip_data.apply(lambda x: x-sensor.iloc[range_start])
            

            # 将当前生成的 Series 与 df_calibre 拼接
            df_calibre = pd.concat([df_calibre.reset_index(drop=True), 
                                        calibre_sensor_snip.reset_index(drop=True)],axis=0)
            
            # 重置一下索引，使得索引是唯一的
            df_calibre.reset_index(drop=True, inplace=True)
            
        df_calibre.columns = [sensor_name]
        sensors_calibre = pd.concat([sensors_calibre.reset_index(drop=True),
                            df_calibre.reset_index(drop=True)],axis=1)

        # 绘图
        # sensor.plot()
        # df_calibre.plot()
        
    sensors_calibre.insert(0,"part", part)

    return sensors_calibre

In [41]:
# 重命名代码
def CalibreFileName(file):
    # 重新构建新文件名
    regex = r"(.*_)(.*_.*)"

    match =  re.search(regex,file)

    # 对容易报错的地方增加 try except
    # 不影响程序运行
    try:
        match = re.search(regex,file)
        if match:
            user = match[1]
            extracted_filename = match[2]
        else:
            print("未匹配到文件名")
    # 能够把 re 的 error 输出出来
    except re.error as e:
        print(f"正则表达式错误:{e}")

    new_file = user + "Calibre-" + extracted_filename
    return new_file

In [39]:
files = os.listdir(Path("./RawData"))

for file in files:
    if ".csv" in file:

        # 读取文件
        file_path = os.path.join(path,"RawData",file)
        df = pd.read_csv(file_path)

        L_data = df[df["part"]=="L"]
        R_data = df[df["part"]=="R"]

        # 对左右鞋垫数据分别进行校准
        L_SensorsCalibre = SensorsCalibre(L_data,"L")
        R_SensorsCalibre = SensorsCalibre(R_data,"R")

        SensorsCalibre_df = pd.concat([L_SensorsCalibre.reset_index(drop=True),
                                    R_SensorsCalibre.reset_index(drop=True)])

        # 重命名
        newfile = CalibreFileName(file)
        new_file_path =  os.path.join(path,DynamicCalibreDir,new_file)

        # 保存文件
        SensorsCalibre_df.to_csv(new_file_path,index=False)